<a href="https://colab.research.google.com/github/shizoda/education/blob/main/info_theory/communication_receiver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 通信シミュレーション（受信側）

このプログラムは，情報理論における符号化を学ぶため，雑音のある通信路を経由したメッセージを受信するためのプログラムです．具体的には，雑音のある通信路を経由したデータを受信し，ハフマン復号化とハミング復号化を行い，受信したデータを正しく読み取ります．また，今回は情報のやりとりをGoogle Driveを経由して行います．

## 準備

まず，Google Driveをマウントします．この際，Googleからの警告が出るので，マウントの許可を行ってください．このとき，送信するデータはMyDriveの下に自動で追加されるinfo-theorフォルダに保存されます．

使用するライブラリはGithub上に存在するため，wgetで取得し，importを行います．

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab")
    drive.mount('/content/drive')
    saved_dir = '/content/drive/MyDrive/info-theory'
except ImportError:
    IN_COLAB = False
    print("Not running on Google Colab")
    saved_dir = '//nas/info-theory'

import os
os.makedirs(saved_dir, exist_ok=True)
print(f"Saved directory is set to: {saved_dir}")

print("current directory", os.getcwd())
import sys
sys.path.append("/content/drive/MyDrive")
# GitHubリポジトリのURL
!wget -q https://raw.githubusercontent.com/shizoda/education/main/info_theory/communication_lib.py

# 必要なPythonファイルをインポートする
from communication_lib import  *

Running on Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved directory is set to: /content/drive/MyDrive/info-theory
current directory /content


## メッセージ

送り主，宛先，メッセージ本文を指定します．以下のコードでは，aliceがデータを送信し，bobが受信します．

In [ ]:
message_from = "alice"
message_to   = "bob"

# 相手から送信されたビットを受信
data_filename = f"{saved_dir}/{message_from}-{message_to}.txt"
codebook_filename = f"{saved_dir}/{message_from}-{message_to}-codebook.json"
padding_filename = f"{saved_dir}/{message_from}-{message_to}-padding.txt"

In [ ]:
codebook, padding_length, received_data = receive_message(codebook_filename, padding_filename, data_filename, message_from, message_to)

## 受信・復号化の実施

以下ではデータの受信，ハフマン復号化，ハミング復号化を行います． 次の説明を参考に，データの受信，ハフマン復号化，ハミング復号化を行うセルの順番を正しく入れ替えて，データを受信してください．

### 雑音のある通信路からの受信

実際にデータをやりとりする際には，ハードウェアの影響や信号の劣化などにより，0と1が反転するなど，データが書き換わることがあります．以下では，単一ビットについてビットの反転エラーが起こったデータをGoogle Driveから受信します．

### ハフマン符号

ハフマン符号は，可逆なデータ圧縮のためのアルゴリズムの1つです．基本原理としては，出現頻度の高い文字に対しては短い符号を，頻度の高い文字に対しては長い符号を振り分けることで，すべての文字に同じ長さの符号を振り分けるよりも，全体の符号化後のデータサイズを最小にしています．

実際にハフマン符号化を行う際には，文字の出現頻度によって，ハフマン木と呼ばれる木構造を作成します．この木では，葉に近いノードほど出現頻度が低く，根に近いノードほど出現頻度が高くなります．また，自分よりも根に近いノードの数が多ければ多いほど，符号の長さが長くなります．

以下の復号化では圧縮されたデータを復元します．

### ハミング符号

ハミング符号は，データ通信などで誤り検出と訂正を行うための誤り訂正符号の一種です．これにより，通信中や保存中に発生する単一ビット誤りを検出し，訂正することが出来ます．ハミング符号は，データビットとパリティビットを組み合わせて構成されます．データビットは送信したいデータが保存され，パリティビットは特定のデータビットの組み合わせに基づいて計算され，データビットに誤りが生じた際に，その位置を特定し，訂正するために使用されます．

ハミング符号は，単一ビットエラーには対応できますが，複数のビットエラーには対応できないため，実用上はより高度なエラー訂正技術と組み合わせて使用されます．

以下の復号化では，実際にエラーが発生したデータの誤りを検出・訂正します．

In [ ]:
print("雑音のある通信路から受信")
print(f"Received      : {format_binary_string(received_data)}")

雑音のある通信路から受信
Received      : 1101100 1001000 1110000 1111011 0010111 1000011 0111100 0...


In [ ]:
print("〇〇〇△△器：□□□□符号")
hamming_decoded_data = hamming_decode(received_data)

〇〇〇△△器：□□□□符号
213 corrections in total.
  (Block-0, Bit-1): 1101100 --> 1001100
  (Block-1, Bit-4): 1001000 --> 1001100
  (Block-3, Bit-4): 1111011 --> 1111111
  (Block-4, Bit-6): 0010111 --> 0010110
  (Block-8, Bit-1): 1001001 --> 1101001
  (Block-10, Bit-2): 0100011 --> 0110011
  (Block-12, Bit-2): 1011100 --> 1001100
  (Block-13, Bit-2): 1011100 --> 1001100
  (Block-14, Bit-2): 0111010 --> 0101010
  (Block-17, Bit-6): 0111101 --> 0111100
      etc.
Received       : 1101100 1001000 1110000 1111011 0010111 1000011 0111100 0...
Hamming Decoded: 1001100 1001100 1110000 1111111 0010110 1000011 0111100 0...
Hamming Output : 0100 0100 1000 1111 1110 0011 1100 1110 0001 1001 1011 1100 01.. .


In [ ]:
print("〇〇〇△△器：□□□□符号")
decoded_message = huffman_decoding(hamming_decoded_data, codebook, padding_length)

〇〇〇△△器：□□□□符号

Codebook:  e:00  :011 i:0101 l:1010 s:1011 n:1110 E:01000 a:01001 c:10000 v:10010 t:10011 y:11000 g:11010 r:11111 p:100010 ,:100011 .:110010 x:110111 o:111100 m:1100110 d:1101100 u:1101101 h:1111010 q:11001110 f:11001111 b:11110110 ':111101110 j:111101111

Huffman decoded: 01000 10010 00 11111 11000 111100 1110 00 011 00 110111 100010 00 10000 10011 10...


## 受信したデータの確認

以下では，受信して復号化したデータを表示します．実際に送信したデータと合っているかを確認してください．

In [ ]:
received_message = decoded_message
print(f"Final Decoded Message: {received_message}")

Final Decoded Message: Everyone expects excellent experiences every evening. Each evening, Ellie eagerly eats exquisite edibles, enjoying every essence. Evenings envelop Ellie, embracing endless energy. Ellie engages effortlessly, expressing emotions endlessly. Every encounter elevates Ellie's enthusiasm, enriching every experience eternally.
